In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("raw_trx_codes.csv", sep="|", dtype='str')

In [3]:
df.columns

Index(['Unnamed: 0', 'TRX_CODE', 'TC_GROUP', 'TC_SUBGROUP', 'D3'], dtype='object')

In [4]:
df

,Unnamed: 0,TRX_CODE,TC_GROUP,TC_SUBGROUP,D3
0,0,8997,PAY,INT-PAY,Deposit Transfer at Check-In
1,1,8998,PAY,INT-PAY,Deposit Balance Forward
2,2,9000,PAY,CASH,Cash
3,3,9001,PAY,CASH,Check NIS
4,4,9002,PAY,DB,City Ledger
...,...,...,...,...,...
112,112,9513,PAY,CCARD,MC USD Pele-Link OPI
113,113,9514,PAY,CCARD,Isracard NIS Pele-Link OPI
114,114,9515,PAY,CCARD,Diners USD Pele-Link OPI
115,115,9516,PAY,CCARD,Diners NIS Pele-Link OPI


In [5]:
# take only needed columns
df = df[["TRX_CODE", "D3"]]

In [6]:
# rename columns for comfort
columns_new_names = ["code", "descr"]
df.columns = columns_new_names

In [7]:
df.columns

Index(['code', 'descr'], dtype='object')

In [8]:
df[df['code'].isna()], df[df['descr'].isna()],

(    code descr
 116  NaN   NaN,
     code descr
 116  NaN   NaN)

In [9]:
# drop NaN
df = df.dropna()

In [10]:
# make column strings lowercase
df['descr'] = df['descr'].apply(lambda x: x.lower() if isinstance(x, str) else x)

In [11]:
df.shape

(116, 2)

In [12]:
# correcting some misspelled words
misspelled_words = ['off line', 'on line', 'pele-link', 'nis ', ' nis ', ' nis', r' \- ', r' \-',
                    'american express', r'^mc', 'master card', 'isrcard', ' isr ', 'dolar-',
                    'diners club', 'pos-mastercard', 'pos-', 'e-shop', 'euro']
correct_words = ['offline', 'online', 'pelelink', 'ils ', ' ils ', ' ils', ' ', ' ',
                 'amex', 'mastercard', 'mastercard', 'mastercard', ' mastercard ', 'usd',
                 'diners', 'pos mastercard', 'pos', 'eshop', 'eur']

def replace_misspelled():
    for misspelled, correct in zip(misspelled_words, correct_words):
        df["descr"] = df["descr"].str.replace(misspelled, correct, regex=True)

replace_misspelled()


In [13]:
# create new column for: currencie(s);
all_usd_variations = [" usd ", " usd", "usd "]
df["usd"] = df["descr"].apply(lambda x: True if any(usd_var in x for usd_var in all_usd_variations) else False)

In [14]:
# create new column for: transaction type(s); 
cards = ["visa", "mastercard", "isracard", "amex", "diners", "union pay"]

In [15]:
df["card"] = df["descr"].apply(lambda x: True if any(card in x for card in cards) else False)
df["online"] = df["descr"].apply(lambda x: True if "online" in x else False)
df["pos"] = df["descr"].apply(lambda x: True if "deposit" not in x and "pos" in x else False)
df["parking"] = df["descr"].apply(lambda x: True if "parking" in x else False)

# other way of doing it but using numpy (import numpy as np) :
# df["in_usd"] = np.where(df["descr"].str.contains(" usd "), True, False)

In [16]:
trx_types: list[str] = ['visa', 'mastercard', 'amex', 'diners', 'union pay', 'cash', 'check']
for trx in trx_types:
    df.loc[df['descr'].str.contains(trx), 'short_descr'] = trx

In [17]:
df['short_descr'] = df['short_descr'].fillna('other')

In [18]:
# get rid of redundant words in description columns
# df["descr"] = df["descr"].str.replace(r"/[ ]*online[ ]*/", " ", regex=True)
# df["descr"] = df["descr"].str.replace(r"[ ]*ils[ ]*|[ ]*usd[ ]*", " ", regex=True)
# df["descr"] = df["descr"].str.replace(r"[ ]*pos[ ]+", "", regex=True)
# df["descr"] = df["descr"].str.replace(r"[ ]*offline[ ]*", "", regex=True)

In [19]:
# trim whitespaces
df['descr'] = df['descr'].apply(lambda x: x.strip() if isinstance(x, str) else x)
df['short_descr'] = df['short_descr'].apply(lambda x: x.strip() if isinstance(x, str) else x)

In [20]:
df,

(     code                         descr    usd   card  online    pos  parking  \
 0    8997  deposit transfer at check-in  False  False   False  False    False   
 1    8998       deposit balance forward  False  False   False  False    False   
 2    9000                          cash  False  False   False  False    False   
 3    9001                     check ils  False  False   False  False    False   
 4    9002                   city ledger  False  False   False  False    False   
 ..    ...                           ...    ...    ...     ...    ...      ...   
 111  9512         amex ils pelelink opi  False   True   False  False    False   
 112  9513   mastercard usd pelelink opi   True   True   False  False    False   
 113  9514     isracard ils pelelink opi  False   True   False  False    False   
 114  9515       diners usd pelelink opi   True   True   False  False    False   
 115  9516       diners ils pelelink opi  False   True   False  False    False   
 
     short_des

In [21]:
df_cards_columns = ['code', 'short_descr', 'usd', 'online', 'pos', 'parking', 'descr']

In [22]:
df_cards = df.loc[df["card"]]

In [23]:
df_cards = df_cards[df_cards_columns]
df_cards, df_cards.shape

(     code short_descr    usd  online    pos  parking  \
 5    9003        amex   True   False  False    False   
 6    9004        visa   True   False  False    False   
 7    9005  mastercard   True   False  False    False   
 8    9006      diners   True   False  False    False   
 11   9011        amex  False   False  False    False   
 ..    ...         ...    ...     ...    ...      ...   
 111  9512        amex  False   False  False    False   
 112  9513  mastercard   True   False  False    False   
 113  9514       other  False   False  False    False   
 114  9515      diners   True   False  False    False   
 115  9516      diners  False   False  False    False   
 
                            descr  
 5               amex usd offline  
 6               visa usd offline  
 7         mastercard usd offline  
 8             diners usd offline  
 11              amex ils offline  
 ..                           ...  
 111        amex ils pelelink opi  
 112  mastercard usd pelel

In [24]:
df_visa = df_cards.loc[df_cards['short_descr'] == 'visa']
df_visa, df_visa.shape

(     code short_descr    usd  online    pos  parking  \
 6    9004        visa   True   False  False    False   
 12   9012        visa  False   False  False    False   
 19   9022        visa  False   False  False    False   
 32   9122        visa   True   False  False    False   
 33   9123        visa  False   False  False    False   
 41   9201        visa   True   False  False    False   
 45   9205        visa  False   False  False    False   
 50   9210        visa   True   False  False    False   
 54   9214        visa  False   False  False    False   
 58   9218        visa   True   False  False    False   
 59   9219        visa  False   False  False    False   
 62   9250        visa  False   False  False     True   
 70   9304        visa  False   False   True    False   
 82   9351        visa  False   False   True    False   
 86   9355        visa   True   False   True    False   
 92   9403        visa   True    True  False    False   
 93   9404        visa  False  

In [25]:
df_mastercard = df_cards.loc[df_cards['short_descr'] == 'mastercard']
df_mastercard, df_mastercard.shape

(     code short_descr    usd  online    pos  parking  \
 7    9005  mastercard   True   False  False    False   
 15   9015  mastercard  False   False  False    False   
 21   9025  mastercard  False   False  False    False   
 36   9126  mastercard   True   False  False    False   
 42   9202  mastercard   True   False  False    False   
 46   9206  mastercard  False   False  False    False   
 48   9208  mastercard  False   False  False    False   
 51   9211  mastercard   True   False  False    False   
 55   9215  mastercard  False   False  False    False   
 57   9217  mastercard  False   False  False    False   
 60   9220  mastercard   True   False  False    False   
 61   9221  mastercard  False   False  False    False   
 63   9251  mastercard  False   False  False     True   
 71   9305  mastercard  False   False   True    False   
 73   9307  mastercard  False   False   True    False   
 87   9356  mastercard   True   False   True    False   
 94   9405  mastercard   True  

In [26]:
df_amex = df_cards.loc[df['short_descr'] == 'amex']
df_amex, df_amex.shape

(     code short_descr    usd  online    pos  parking  \
 5    9003        amex   True   False  False    False   
 11   9011        amex  False   False  False    False   
 18   9021        amex  False   False  False    False   
 34   9124        amex   True   False  False    False   
 35   9125        amex  False   False  False    False   
 40   9200        amex   True   False  False    False   
 44   9204        amex  False   False  False    False   
 49   9209        amex   True   False  False    False   
 53   9213        amex  False   False  False    False   
 65   9253        amex  False   False  False     True   
 69   9303        amex  False   False   True    False   
 81   9350        amex  False   False   True    False   
 85   9354        amex   True   False   True    False   
 90   9401        amex   True    True  False    False   
 91   9402        amex  False    True  False    False   
 101  9502        amex   True   False  False    False   
 102  9503        amex  False  

In [27]:
df_diners = df_cards.loc[df_cards['short_descr'] == 'diners']
df_diners, df_diners.shape

(     code short_descr    usd  online    pos  parking  \
 8    9006      diners   True   False  False    False   
 14   9014      diners  False   False  False    False   
 20   9024      diners  False   False  False    False   
 38   9128      diners   True   False  False    False   
 39   9129      diners  False   False  False    False   
 43   9203      diners   True   False  False    False   
 47   9207      diners  False   False  False    False   
 52   9212      diners   True   False  False    False   
 56   9216      diners  False   False  False    False   
 64   9252      diners  False   False  False     True   
 72   9306      diners  False   False   True    False   
 84   9353      diners  False   False   True    False   
 88   9357      diners   True   False   True    False   
 97   9408      diners   True    True  False    False   
 98   9409      diners  False    True  False    False   
 105  9506      diners   True   False  False    False   
 106  9507      diners  False  

In [28]:
# check what else was left behind
# ~  means NOT IN
df_cards.loc[
    ~(
        df_cards["descr"].isin(df_visa["descr"]) | 
        df_cards["descr"].isin(df_mastercard["descr"]) | 
        df_cards["descr"].isin(df_amex["descr"]) | 
        df_cards["descr"].isin(df_diners["descr"]) 
    )
],


(     code short_descr    usd  online    pos  parking  \
 16   9019   union pay   True   False  False    False   
 17   9020   union pay  False   False  False    False   
 37   9127       other  False   False  False    False   
 83   9352       other  False   False   True    False   
 96   9407       other  False    True  False    False   
 104  9505       other  False   False  False    False   
 113  9514       other  False   False  False    False   
 
                          descr  
 16               union pay usd  
 17               union pay ils  
 37    isracard ils sfc deposit  
 83            pos isracard ils  
 96         isracard ils online  
 104      isracard ils pelelink  
 113  isracard ils pelelink opi  ,)

In [29]:
df_union_pay = df_cards.loc[df_cards['short_descr'] == 'union pay']
df_union_pay, df_union_pay.shape

(    code short_descr    usd  online    pos  parking          descr
 16  9019   union pay   True   False  False    False  union pay usd
 17  9020   union pay  False   False  False    False  union pay ils,
 (2, 7))

In [30]:
df_pos = df_cards.loc[df_cards["pos"]]
df_pos, df_pos.shape

(    code short_descr    usd  online   pos  parking               descr
 69  9303        amex  False   False  True    False        pos amex ils
 70  9304        visa  False   False  True    False            pos visa
 71  9305  mastercard  False   False  True    False      pos mastercard
 72  9306      diners  False   False  True    False          pos diners
 73  9307  mastercard  False   False  True    False  pos mastercard ils
 81  9350        amex  False   False  True    False        pos amex ils
 82  9351        visa  False   False  True    False        pos visa ils
 83  9352       other  False   False  True    False    pos isracard ils
 84  9353      diners  False   False  True    False      pos diners ils
 85  9354        amex   True   False  True    False        pos amex usd
 86  9355        visa   True   False  True    False        pos visa usd
 87  9356  mastercard   True   False  True    False  pos mastercard usd
 88  9357      diners   True   False  True    False      pos din

In [31]:
df_offline = df_cards.loc[df_cards["online"] == False]
df_offline, df_offline.shape

(     code short_descr    usd  online    pos  parking  \
 5    9003        amex   True   False  False    False   
 6    9004        visa   True   False  False    False   
 7    9005  mastercard   True   False  False    False   
 8    9006      diners   True   False  False    False   
 11   9011        amex  False   False  False    False   
 ..    ...         ...    ...     ...    ...      ...   
 111  9512        amex  False   False  False    False   
 112  9513  mastercard   True   False  False    False   
 113  9514       other  False   False  False    False   
 114  9515      diners   True   False  False    False   
 115  9516      diners  False   False  False    False   
 
                            descr  
 5               amex usd offline  
 6               visa usd offline  
 7         mastercard usd offline  
 8             diners usd offline  
 11              amex ils offline  
 ..                           ...  
 111        amex ils pelelink opi  
 112  mastercard usd pelel

In [32]:
df_online = df_cards.loc[df_cards["online"]]
df_online, df_online.shape

(    code short_descr    usd  online    pos  parking                  descr
 90  9401        amex   True    True  False    False        amex usd online
 91  9402        amex  False    True  False    False        amex ils online
 92  9403        visa   True    True  False    False        visa usd online
 93  9404        visa  False    True  False    False        visa ils online
 94  9405  mastercard   True    True  False    False  mastercard usd online
 95  9406  mastercard  False    True  False    False  mastercard ils online
 96  9407       other  False    True  False    False    isracard ils online
 97  9408      diners   True    True  False    False      diners usd online
 98  9409      diners  False    True  False    False      diners ils online,
 (9, 7))

In [33]:
# check if anything left to sort out
# ~  means NOT IN
df_other = df.loc[~df["descr"].isin(df_cards["descr"])]
df_other, df_other.shape

(    code                         descr    usd   card  online    pos  parking  \
 0   8997  deposit transfer at check-in  False  False   False  False    False   
 1   8998       deposit balance forward  False  False   False  False    False   
 2   9000                          cash  False  False   False  False    False   
 3   9001                     check ils  False  False   False  False    False   
 4   9002                   city ledger  False  False   False  False    False   
 9   9007             delayed check ils  False  False   False  False    False   
 10  9008                     check usd   True  False   False  False    False   
 13  9013     do not use !!- go to 9206  False  False   False  False    False   
 22  9030             direct settlement  False  False   False  False    False   
 23  9100    bank benleumi/hapoalim usd   True  False   False  False    False   
 24  9101    bank benleumi/hapoalim ils  False  False   False  False    False   
 25  9102                ban

In [34]:
# df_cards.to_csv("cards.csv", sep="|", index=False, encoding="utf-8")

In [35]:
# df_visa.to_csv("cards_visa.csv", sep="|", index=False, encoding="utf-8")

In [36]:
# df_mastercard.to_csv("cards_mastercard.csv", sep="|", index=False, encoding="utf-8")

In [37]:
# df_amex.to_csv("cards_amex.csv", sep="|", index=False, encoding="utf-8")

In [38]:
# df_diners.to_csv("cards_diners.csv", sep="|", index=False, encoding="utf-8")

In [39]:
# df_pos.to_csv("cards_pos.csv", sep="|", index=False, encoding="utf-8")

In [40]:
# df_online.to_csv("cards_online.csv", sep="|", index=False, encoding="utf-8")

In [41]:
# df_offline.to_csv("cards_offline.csv", sep="|", index=False, encoding="utf-8")